pip install opencv-python
pip install imutils
pip install dlib

In [ ]:
from imutils.face_utils import FaceAligner, rect_to_bb
import imutils
import dlib
import cv2
import matplotlib.pyplot as plt
from glob import glob
import os
import requests
from datetime import datetime
import json
from PIL import Image, ImageOps
from pillow_heif import register_heif_opener

register_heif_opener()

In [ ]:
# capture issues to resolve in the future:
def load_array_from_file(filename):
    with open(filename, 'r') as f:
        array = [line.strip() for line in f]
    print(f"Data loaded from {filename}")
    return array

def save_array_to_file(array, filename):
    with open(filename, 'a') as f:
        for item in array:
            f.write(f"{item}\n")
    print(f"Data saved to {filename}")

capture_issues = load_array_from_file('capture_issues.txt')
need_to_be_renamed = load_array_from_file('needs_to_be_renamed.txt')
script_failures = load_array_from_file('script_failures.txt')

image_issues = capture_issues + need_to_be_renamed + script_failures

# mapping if first rectangle isn't ian
rectangle_mapping  = json.load(open('rectangle_mapping.json'))

In [ ]:
from imutils.face_utils.helpers import FACIAL_LANDMARKS_68_IDXS
from imutils.face_utils.helpers import FACIAL_LANDMARKS_5_IDXS
from imutils.face_utils.helpers import shape_to_np
import numpy as np
import cv2

class FaceAligner:
	def __init__(self, predictor, desiredLeftEye=(0.35, 0.35),
		desiredFaceWidth=256, desiredFaceHeight=None):
		# store the facial landmark predictor, desired output left
		# eye position, and desired output face width + height
		self.predictor = predictor
		self.desiredLeftEye = desiredLeftEye
		self.desiredFaceWidth = desiredFaceWidth
		self.desiredFaceHeight = desiredFaceHeight

		# if the desired face height is None, set it to be the
		# desired face width (normal behavior)
		if self.desiredFaceHeight is None:
			self.desiredFaceHeight = self.desiredFaceWidth

	def align(self, image, gray, rect):
		# convert the landmark (x, y)-coordinates to a NumPy array
		shape = self.predictor(gray, rect)
		shape = shape_to_np(shape)
		
		#simple hack ;)
		if (len(shape)==68):
			# extract the left and right eye (x, y)-coordinates
			(lStart, lEnd) = FACIAL_LANDMARKS_68_IDXS["left_eye"]
			(rStart, rEnd) = FACIAL_LANDMARKS_68_IDXS["right_eye"]
		else:
			(lStart, lEnd) = FACIAL_LANDMARKS_5_IDXS["left_eye"]
			(rStart, rEnd) = FACIAL_LANDMARKS_5_IDXS["right_eye"]
			
		leftEyePts = shape[lStart:lEnd]
		rightEyePts = shape[rStart:rEnd]

		# compute the center of mass for each eye
		leftEyeCenter = leftEyePts.mean(axis=0).astype("int")
		rightEyeCenter = rightEyePts.mean(axis=0).astype("int")

		# compute the angle between the eye centroids
		dY = rightEyeCenter[1] - leftEyeCenter[1]
		dX = rightEyeCenter[0] - leftEyeCenter[0]
		angle = np.degrees(np.arctan2(dY, dX)) - 180

		# compute the desired right eye x-coordinate based on the
		# desired x-coordinate of the left eye
		desiredRightEyeX = 1.0 - self.desiredLeftEye[0]

		# determine the scale of the new resulting image by taking
		# the ratio of the distance between eyes in the *current*
		# image to the ratio of distance between eyes in the
		# *desired* image
		dist = np.sqrt((dX ** 2) + (dY ** 2))
		desiredDist = (desiredRightEyeX - self.desiredLeftEye[0])
		desiredDist *= self.desiredFaceWidth
		scale = desiredDist / dist

		# compute center (x, y)-coordinates (i.e., the median point)
		# between the two eyes in the input image
		eyesCenter = (int((leftEyeCenter[0] + rightEyeCenter[0]) // 2),
            int((leftEyeCenter[1] + rightEyeCenter[1]) // 2))

		# grab the rotation matrix for rotating and scaling the face
		M = cv2.getRotationMatrix2D(eyesCenter, angle, scale)

		# update the translation component of the matrix
		tX = self.desiredFaceWidth * 0.5
		tY = self.desiredFaceHeight * self.desiredLeftEye[1]
		M[0, 2] += (tX - eyesCenter[0])
		M[1, 2] += (tY - eyesCenter[1])

		# apply the affine transformation
		(w, h) = (self.desiredFaceWidth, self.desiredFaceHeight)
		output = cv2.warpAffine(image, M, (w, h),
			flags=cv2.INTER_CUBIC)

		# return the aligned face
		return output

In [ ]:
landmark_name = "shape_predictor_68_face_landmarks.dat"
if not os.path.isfile(landmark_name):
    landmarks_url = "https://github.com/italojs/facial-landmarks-recognition/raw/master/shape_predictor_68_face_landmarks.dat"
    out = requests.get(landmarks_url)
    open(landmark_name, "wb").write(out.content)


In [ ]:
# get paths
paths = glob('original_faces/*')
# remove MP$
paths = [path for path in paths if not path.endswith('MP4')]
failed_paths = []
verbose = False
save = True

size = 1024

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')
fa = FaceAligner(predictor, desiredFaceWidth=size, desiredLeftEye=(.42,.5))

def to_aligned_path(image_path):
    aligned_base = f'aligned_faces/{os.path.basename(image_path)}'
    aligned_path =  os.path.splitext(aligned_base)[0] + '.jpg'
    return aligned_path

for image_path in paths:
    aligned_path = to_aligned_path(image_path)
    base_path = os.path.basename(image_path)
    if os.path.isfile(aligned_path) or base_path in image_issues:
        continue
    try:
        # load the input image, resize it, and convert it to grayscale
        # load and transform image
        image = Image.open(image_path).convert('RGB')
        image = ImageOps.exif_transpose(image)
        image = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
        image = imutils.resize(image, width=800)
        print(f'successful load of {image_path}')
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        # show the original input image and detect faces in the grayscale
        # image
        cv2.imshow("Input", image)
        rects = detector(gray, 2)

        # get correct face detection
        rect = rects[rectangle_mapping.get(base_path, 0)]
        # extract the ROI of the *original* face, then align the face
        # using facial landmarks
        (x, y, w, h) = rect_to_bb(rect)
        faceOrig = imutils.resize(image[y:y + h, x:x + w], width=size)
        faceAligned = fa.align(image, gray, rect)
        if save:
            cv2.imwrite(aligned_path, faceAligned)
        # display the output images
        if verbose:
            plt.subplot(1,2,1)
            plt.imshow(faceOrig)
            plt.subplot(1,2,2)
            plt.imshow(faceAligned)
            plt.show()
    except Exception as e:
        print(e)
        print(f'failed to run {image_path}')
        failed_paths.append(image_path)

failed_base_paths = [os.path.basename(path) for path in failed_paths]
save_array_to_file(failed_base_paths, 'script_failures.txt')
image_issues += failed_base_paths

In [ ]:
# verbose=True
# image_path = "original_faces/IMG_7656.HEIC"
# base_path = os.path.basename(image_path)

# # load the input image, resize it, and convert it to grayscale
# # load and transform image
# image = Image.open(image_path).convert('RGB')
# image = ImageOps.exif_transpose(image)
# image = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
# image = imutils.resize(image, width=800)
# print(f'successful load of {image_path}')
# gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
# # show the original input image and detect faces in the grayscale
# # image
# cv2.imshow("Input", image)
# rects = detector(gray, 2)

# # get correct face detection
# rect = rects[rectangle_mapping.get(base_path, 1)]
# # extract the ROI of the *original* face, then align the face
# # using facial landmarks
# (x, y, w, h) = rect_to_bb(rect)
# faceOrig = imutils.resize(image[y:y + h, x:x + w], width=size)
# faceAligned = fa.align(image, gray, rect)
# if save:
#     cv2.imwrite(aligned_path, faceAligned)
# # display the output images
# if verbose:
#     plt.subplot(1,2,1)
#     plt.imshow(faceOrig)
#     plt.subplot(1,2,2)
#     plt.imshow(faceAligned)
#     plt.show()

In [ ]:
def get_date_taken(path):
    exif = Image.open(path).getexif()
    if not exif:
        print('Image {0} does not have EXIF data.'.format(path))
        return None
    return exif.get(306)

def add_date_to_image(image, date_string):
    # Convert date string to desired format
    date_obj = datetime.strptime(date_string, "%Y:%m:%d %H:%M:%S")
    formatted_date = date_obj.strftime("%Y-%m-%d")
    
    # Set text parameters
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 1
    font_color = (255, 255, 255)  # White color
    thickness = 2
    
    # Get text size
    text_size = cv2.getTextSize(formatted_date, font, font_scale, thickness)[0]
    
    # Set text position (bottom left)
    text_x = 10
    text_y = image.shape[0] - 10  # 10 pixels from the bottom
    
    # Add black background for better readability
    cv2.rectangle(image, (text_x, text_y - text_size[1] - 10),
                  (text_x + text_size[0] + 10, text_y + 10),
                  (0, 0, 0), -1)
    
    # Add text to the image
    cv2.putText(image, formatted_date, (text_x + 5, text_y), 
                font, font_scale, font_color, thickness)
    
    return image

dates = []
for path in paths:
    if os.path.basename(path) in image_issues:
        continue
    date = get_date_taken(path)
    if date is not None:
        dates.append((date, to_aligned_path(path)))
# sort the dates
dates = sorted(dates, key = lambda x: x[0])

# get all image paths
image_paths = [i[1] for i in dates]
frame = cv2.imread(image_paths[0])
height, width, layers = frame.shape


In [ ]:
# create a video
def create_video(output_file_name, fps, date_img_array, add_image=True):
    fourcc = cv2.VideoWriter_fourcc(*'MP4V')
    video = cv2.VideoWriter(output_file_name, fourcc, fps, (width,height))
    for date, path in date_img_array:
        img = cv2.imread(path)
        if add_image:
            img_with_date = add_date_to_image(img, date)
            video.write(img_with_date)
        else:
            video.write(img)

    cv2.destroyAllWindows()
    video.release()

output_filename = 'videos/aligned_faces_video.mp4'
fps = 9
create_video(output_filename, fps, dates)

In [ ]:
# Create average photo
def create_average_image(image_paths):        
    average_image = None
    total_images = 0
    # Iterate over the rest of the images
    for path in image_paths:
        # Read the image
        image = cv2.imread(path)
        try:
            image = image.astype(np.float32)
            total_images += 1
        except:
            continue
        # Add the image to the average
        if average_image is None:
            average_image = image
        else:
            average_image += image
    if average_image is not None:
        # Divide by the number of images to get the average
        average_image /= total_images

        # Convert back to 8-bit
        average_image = average_image.astype(np.uint8)
        return average_image

average_image = create_average_image(image_paths)
# Save the averaged image
cv2.imwrite('average_images/overall_average_image.jpg', average_image)

In [ ]:
from datetime import datetime
from collections import defaultdict
images_by_year = defaultdict(list)
images_by_month = defaultdict(list)
images_by_quarter = defaultdict(list)
# Iterate over the dates and images
for date_str, image_path in dates:
    # Parse the date
    date = datetime.strptime(date_str, '%Y:%m:%d %H:%M:%S')

    # Calculate the quarter
    quarter = (date.month - 1) // 3 + 1

    year_key = date.year
    month_key = (date.year, date.month)
    quarter_key = (date.year, f"Q{quarter}")
    
    # Add the image to the appropriate list
    images_by_year[year_key].append(image_path)
    images_by_month[month_key].append(image_path)
    images_by_quarter[quarter_key].append(image_path)

    
for key, images in images_by_quarter.items():
    # Create the output filename
    output_filename = 'average_images/{0}_{1}_average_image_{2}images.jpg'.format(key[0], key[1], len(images))
    
    # Create the average image for the month
    average_image = create_average_image(images)
    if average_image is not None:
        # Save the image
        cv2.imwrite(output_filename, average_image)

In [ ]:
for key, images in images_by_year.items():
    date_subset = [i for i in dates if i[1] in images]
    output_filename = f'videos/{key}_aligned_faces_video.mp4'
    if len(date_subset) > 50:
        create_video(output_filename, fps, date_subset)


## Future Explorations
URL: https://archive.is/lQXdx